<a href="https://colab.research.google.com/github/ThibeVanOrshaegen/.github/blob/main/Oefeningzeven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile oefeningzeven.cu
#include <iostream>
#include <cuda_runtime.h>

const int N = 1000000; // Aantal elementen in de array

// Kernel om elke waarde in de array te verdubbelen
__global__ void doubleArrayKernel(int *array) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < N) {
        array[tid] *= 2;
    }
}

int main() {
    int *h_array, *d_array;
    size_t size = N * sizeof(int);

    // Host array initialiseren
    h_array = new int[N];
    for (int i = 0; i < N; ++i) {
        h_array[i] = i;
    }

    // Device array initialiseren
    cudaMalloc(&d_array, size);
    cudaMemcpy(d_array, h_array, size, cudaMemcpyHostToDevice);

    // Kernelconfiguratie
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;

    // CUDA events voor timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Start event opnemen
    cudaEventRecord(start);

    // Kernel uitvoeren om de array te verdubbelen
    doubleArrayKernel<<<numBlocks, blockSize>>>(d_array);

    // Stop event opnemen
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Bereken de doorlooptijd
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "Tijd voor kerneluitvoering: " << milliseconds << " ms" << std::endl;

    // Resultaat terug naar de host kopiëren en controleren
    cudaMemcpy(h_array, d_array, size, cudaMemcpyDeviceToHost);
    for (int i = 0; i < 10; ++i) {
        std::cout << "Waarde op index " << i << ": " << h_array[i] << std::endl;
    }

    // Geheugen vrijgeven
    delete[] h_array;
    cudaFree(d_array);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Writing oefeningzeven.cu


In [2]:
!nvcc oefeningzeven.cu -o oefeningzeven

In [3]:
!./oefeningzeven

Tijd voor kerneluitvoering: 119.934 ms
Waarde op index 0: 0
Waarde op index 1: 2
Waarde op index 2: 4
Waarde op index 3: 6
Waarde op index 4: 8
Waarde op index 5: 10
Waarde op index 6: 12
Waarde op index 7: 14
Waarde op index 8: 16
Waarde op index 9: 18
